In [1]:
import os

In [2]:
%pwd

'/home/joshua/Desktop/Refactory/SwahiliNewsClassifier/research'

In [3]:
os.chdir('../')
%pwd

'/home/joshua/Desktop/Refactory/SwahiliNewsClassifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path

from swahiliNewsClassifier.constants import *
from swahiliNewsClassifier.utils.common import read_yaml, create_directories

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from swahiliNewsClassifier import logger

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        """
        Initialize DataIngestion object with the provided configuration.

        Args:
            config (DataIngestionConfig): Configuration object for data ingestion.
        """
        self.config = config

    def download_file(self):
        """Fetch data from a URL.
        
        Raises:
            Exception: If an error occurs during the download process.
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir =  self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """Extract a zip file.

            This method extracts the contents of a zip file specified in the configuration
            to the directory specified in the configuration.

            Raises:
                Exception: If an error occurs during the extraction process.
            """
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)

            with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
                zip_ref.extractall(unzip_path)

            logger.info(f"Extracted zip file into: {unzip_path}")

        except Exception as e:
            logger.error(f"Error extracting zip file: {self.config.local_data_file}")
            raise e
        

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-08 13:15:03,524: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-05-08 13:15:03,538: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-08 13:15:03,543: INFO: common: Created directory at: artifacts]
[2024-05-08 13:15:03,548: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-05-08 13:15:03,557: INFO: 149356000: Downloading data from https://drive.google.com/file/d/15stuLDZkXNOgBUC1rnx5yXYdVPViUjNB/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=15stuLDZkXNOgBUC1rnx5yXYdVPViUjNB
To: /home/joshua/Desktop/Refactory/SwahiliNewsClassifier/artifacts/data_ingestion/data.zip
100%|██████████| 3.78M/3.78M [00:32<00:00, 115kB/s] 

[2024-05-08 13:15:42,626: INFO: 149356000: Downloaded data from https://drive.google.com/file/d/15stuLDZkXNOgBUC1rnx5yXYdVPViUjNB/view?usp=sharing into file artifacts/data_ingestion/data.zip]


[2024-05-08 13:15:43,091: INFO: 149356000: Extracted zip file into: artifacts/data_ingestion]
